# Get cuisine data

Description: This file retrieves information from dishes of each cuisine and exports a file with all cuisine data.

In [1]:
# Set up
import urllib.request
import os
from bs4 import BeautifulSoup as bs
from nltk.tokenize import word_tokenize
import string
import statistics
import csv
from pathlib import Path

url = "https://www.allrecipes.com/cuisine-a-z-6740455"
fhand = urllib.request.urlopen(url)
html = fhand.read()
soup = bs(html, "html.parser")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Get cuisine URLs
cuisine_blocks = soup.find_all("div", {"class": "alphabetical-list__group"})
cuisine_urls = set()

for block in cuisine_blocks:
    cuisine_in_block = block.find_all(class_ = "comp link-list__item")
    
    # Loop through each cuisine in the block
    for cuisine in cuisine_in_block:
        cuisine_url = cuisine.find("a")["href"]
        cuisine_urls.add(cuisine_url)

cuisine_urls.add("https://www.allrecipes.com/recipes/728/world-cuisine/latin-american/mexican/")

print(len(cuisine_urls))
        
# Initiate cuisine stats
cuisines_data = dict()

50


In [3]:
# Cuisine helper functions

# Retrieve dish URLs
def get_dish_url():
    dish_urls = []
    a_elem = cuisine_soup.find_all("a", class_ = "comp mntl-card-list-items mntl-document-card mntl-card card card--no-image")
        
    for elem in a_elem:
        dish_url = elem.get("href")
        dish_urls.append(dish_url)
    
    return dish_urls

# Retrieve cuisine names
def get_cuisine_name():
    cuisine_name = cuisine_soup.find("h1", {"class": "comp mntl-taxonomysc-heading mntl-text-block"}).text.split()
    return cuisine_name[0]

# Retrieve dish name
def get_dish_name(dish_soup):
    dish_name = dish_soup.find("h1", {"id": "article-heading_1-0", "class": "comp type--lion article-heading mntl-text-block"}).text.strip()
    return dish_name

# Retrieve number of reviews
def get_num_reviews(dish_soup):
    review_element = dish_soup.find('div', {'id': 'mntl-recipe-review-bar__comment-count_1-0', 'class': 'comp type--squirrel-link mntl-recipe-review-bar__comment-count mntl-text-block'})
    if review_element:
        num_reviews_text = review_element.text.strip()
        num_reviews_dish = "".join(char for char in num_reviews_text if char.isdigit())
        return int(num_reviews_dish)
    else:
        return 0
    
# Retrieve dish ratings
def get_dish_ratings(dish_soup):
    dish_ratings = dish_soup.find("div", {"id": "mntl-recipe-review-bar__rating_1-0",
                                          "class": "comp type--squirrel-bold mntl-recipe-review-bar__rating mntl-text-block"})
    if dish_ratings:
        dish_ratings = dish_ratings.text.strip()
        return float(dish_ratings)
    else:
        return 0
    
# Return a list of words or ngrams to a string
def list_to_str(lst):
    s = ''
    for i in range(len(lst)):
        if type(lst[i]) == int or type(lst[i]) == float:
            s += str(lst[i])
        else:
            s += lst[i]

        if i < len(lst)-1:           
            s += ', '
            
    return "{}".format(s)

In [6]:
# Cuisine data

count = 0
for cuisine_url in cuisine_urls:
    # Open cuisine URL
    cuisine_fhand = urllib.request.urlopen(cuisine_url)
    cuisine_html = cuisine_fhand.read()
    cuisine_soup = bs(cuisine_html, "html.parser")

    # Get cuisine name
    cuisine_name = get_cuisine_name()
    
    count += 1
    print(f"{count}: Analyzing {cuisine_name}...")

    # Get dish URLs
    dish_urls = get_dish_url()

    # Analyze for each dish
    total_num_dishes, total_num_reviews = 0, 0
    most_reviews = 0
    dishes_data, all_dishes = [], []
    all_ratings, all_reviews, most_reviewed_dish = [], [], []
    dish_comments = ""

    for dish_url in dish_urls:
        # Open dish URL
        dish_fhand = urllib.request.urlopen(dish_url)
        dish_html = dish_fhand.read()
        dish_soup = bs(dish_html, "html.parser")

        # Number of dishes
        total_num_dishes += 1

        # Get dish name
        dish_name = get_dish_name(dish_soup)
        
        print(f"----{total_num_dishes}: {dish_name}")

        # Update all_dishes list
        all_dishes.append(dish_name)

        # Get reviews data, get number of reviews, update all_reviews list
        num_reviews_dish = get_num_reviews(dish_soup)
        all_reviews.append(num_reviews_dish)
        total_num_reviews += num_reviews_dish

        # Get ratings data, Update all_ratings list
        dish_ratings = get_dish_ratings(dish_soup)
        all_ratings.append(dish_ratings)

        # Update dishes data
        dishes_data.append((dish_name, num_reviews_dish))

    # Most reviewed dish
    most_reviews = max([num_reviews_dish for dish, num_reviews_dish in dishes_data])
    most_reviewed_dish = [dish[0] for dish in dishes_data if dish[1] == most_reviews]
    most_reviewed_dish = list_to_str(most_reviewed_dish)

    # Get median number reviews
    median_reviews = statistics.median(all_reviews)

    # Get median ratings
    median_rating = statistics.median(all_ratings)

    # Turn final lists into strings
    all_dishes = list_to_str(all_dishes)
    all_reviews = list_to_str(all_reviews)
    all_ratings = list_to_str(all_ratings)

    # Add cuisine information to cuisine data
    cuisines_data[cuisine_name] = {"Number of dishes": total_num_dishes,
                                   "Total number of reviews": total_num_reviews,
                                   "Most reviewed dish": most_reviewed_dish,
                                   "Median number of reviews": median_reviews,
                                   "Reviews data": all_reviews,
                                   "Median rating": median_rating,
                                   "Ratings data": all_ratings,
                                   "Dishes": all_dishes}


1: Analyzing Jamaican...
----1: Jamaican Saltfish Fritters (Stamp and Go)
----2: Sharon's Jamaican Fruit Cake
----3: Jamaican Fried Snapper
----4: Perfect Baked Jerk Chicken
----5: Jerk Marinade Seasoning Rub
----6: Jerk Chicken Wings
----7: Jamaican Cabbage
----8: Jerk Chicken and Pasta
----9: Jamaican Jerk Chicken
----10: Jerk Seasoning
----11: Hummingbird
----12: Jamaican Plantain Tarts
----13: Jamaican Rice and Peas
----14: Hard Do Bread
----15: Jamaican Oxtail with Broad Beans
----16: Jamaican Curry Powder
----17: Jamaican Curry Chicken
----18: Jamaican Curried Goat
----19: We Be Jammin' Jamaican Banana Bread
----20: Boscobel Beach Ginger Cake
----21: Jamaican Cornmeal Porridge
----22: Chef John's Hummingbird Cake
----23: Jerk Pork Chops
----24: Instant Pot® Jamaican Chicken Curry
----25: Runaway Bay Jamaican Chicken
----26: Jamaican Jerked Chicken
----27: Jamaican Oxtail Stew
----28: Hot Banana Salsa
----29: Tanya's Jamaican Spice Bread
----30: A Jerky Chicken
----31: Jerk Chicke

----45: Sushi Party
----46: Milk Flitcher Pie
----47: Simple Stovetop Amish Chicken
----48: 13 Heartwarming Amish Breakfasts
----49: Amish Chocolate Cake
----50: Healthier Amish Friendship Bread Starter
----51: Amish Buttermilk Pie
----52: Red Pepper and Cabbage Pickle
----53: Moist Herman Coffee Cake
----54: Amish Chocolate Pudding
----55: Gera's Amish Funny Cake
----56: Quick Banana Nut Cake
----57: Cherry Amish Friendship Bread Cupcakes with Buttercream Frosting
----58: Keufels (Original)
----59: Amish Brown Sugar Cake
----60: Amish Cran-Apple Pie
6: Analyzing Southern...
----1: 14 Foods Every Southerner Should Have In the Pantry
----2: Sweet Potato Pie
----3: Fresh Southern Peach Cobbler
----4: Best Fried Green Tomatoes
----5: Best Jambalaya
----6: Authentic Louisiana Red Beans and Rice
----7: Southern Pimento Cheese
----8: Southern Coleslaw
----9: Kickin' Collard Greens
----10: Bananas Foster
----11: Chef John's Buttermilk Biscuits
----12: Southern Fried Cabbage
----13: Southern-S

----52: Pierogi (Traditional Polish Dumplings)
----53: Chrusciki
----54: Beer Sausage
----55: Kielbasa
----56: I Finally Found My Grandma's Famous Pierogi Recipe
----57: Jam Kolaczki
----58: Polish Reuben Casserole
----59: Polish Sauerkraut and Carrot Salad
----60: Polish Cabbage Noodles
10: Analyzing Norwegian...
----1: Norwegian Potato Lefsa
----2: Pannekaken (Norwegian Pancakes)
----3: Potato Klubb (Norwegian Potato Dumplings)
----4: Norwegian Butter Sauce (Sandefjordsmor)
----5: Original Kumla Recipe from Mom
----6: Kransekake (Norwegian Almond Ring Cake)
----7: Norwegian Potato Flatbread (Lefse)
----8: Fattigmann
----9: Fårikål
----10: Norwegian Potato Klub
----11: Torsk (Scandinavian Cod)
----12: Bjarne's Norwegian Meatballs
----13: Scandinavian Sweetheart Waffles
----14: Scandinavian Christmas Cookies Bring Old-World Tradition to Your Holiday
----15: Lefse
----16: Norwegian Christmas Cabbage
----17: Norwegian Almond Cake
----18: Kringla II
----19: Kringla
----20: Krumkake II
---

----21: Pakistani Batter-Fried Potatoes
----22: Coconut Chikki
----23: Balti Chicken Pasanda
----24: Pakistani Potato Chicken
----25: Spicy Potato Noodles (Bataka Sev)
----26: Spinach and Cauliflower Bhaji
16: Analyzing Portuguese...
----1: Easy Shrimp Mozambique
----2: Portuguese Sweet Bread I
----3: Caldo Verde (Portuguese Sausage Kale Soup)
----4: Wine and Garlic Pork (Portuguese Vina Dosh)
----5: Malasadas
----6: Portuguese Shrimp
----7: Portuguese Custard Tarts - Pasteis de Nata
----8: Maria's Portuguese Bacalau
----9: Portuguese Soup
----10: Mom's Portuguese Beef Stew
----11: Portuguese Bean Soup II
----12: Portuguese Cod Fish Casserole
----13: Beefs Portuguese Style
----14: Portuguese Muffins - Bolo Levedo
----15: Portuguese Favas
----16: Shrimp Mozambique
----17: Portuguese Sweet Rice
----18: Portuguese Chourico and Peppers
----19: Portuguese Pork Alentejana
----20: Portuguese Sopas
----21: Quince Paste
----22: Queijadas
----23: Kale and Cabbage Soup
----24: Portuguese Steamed 

----45: Okra, Corn and Tomatoes
----46: Easy Hoppin' John
----47: Teena's Overnight Southern Buttermilk Biscuits
----48: CindyD's Somewhat Southern Fried Chicken
----49: Tanya's Louisiana Southern Fried Chicken
----50: Smothered Collard Greens and Cabbage
----51: Okra Patties
----52: Creole Okra
----53: Kentucky Biscuits
----54: Okra and Tomatoes
----55: Southern Collard Greens
----56: Braised Collard Greens
----57: Garlic Chicken Fried Chicken
----58: Down-Home Black-Eyed Peas
----59: Shrimp and Cheesy Grits with Bacon
----60: Slow Cooker Collard Greens
22: Analyzing Puerto...
----1: Tembleque Puerto Rican Coconut Pudding
----2: Sofrito
----3: Coquito
----4: Puerto Rican Canned Corned Beef Stew
----5: Budin (Puerto Rican Bread Pudding)
----6: Bacalao a la Vizcaina (Basque-Style Codfish Stew)
----7: Gourmet Pastelillos (Meat Pies)
----8: Asopao de Pollo
----9: Puerto Rican Breakfast Custard
----10: Puerto Rican Coquito
----11: Limber de Coco (Coconut Ice)
----12: Habichuelas Guisadas
-

----5: Penang Pork Satay
----6: Chicken Curry Puffs
----7: Malaysian Lady Fingers (Okra)
----8: Chicken with Black Beans
----9: Malaysian Flatbread (Roti Canai) with Curried Lentil Dip
----10: Durian Puree Cheesecake
----11: Malaysian Red Curry Thighs
----12: Malaysian Mango Chicken Curry
----13: Pumpkin Kaya
----14: Chicken Curry V
----15: Sherry Chicken Curry
----16: Chicken Singapore Noodles
----17: Chicken Curry III
----18: Rendang Gaging (Beef Stew)
----19: Malaysian Chicken Curry
----20: Malaysian Chicken Satay
----21: Mild Coconut Tofu Curry
----22: Simple Malaysian Fried Chicken
----23: Spicy Noodles - Malay Style
28: Analyzing French...
----1: Apple Tarte Tatin
----2: Provencal White Beans
----3: Sole Meuniere
----4: Pain au Chocolat (Chocolate Croissants)
----5: Salmon en Croûte
----6: Meat Pie
----7: Sauce Vierge
----8: Beans and Greens Tartine
----9: Pâte Sucrée
----10: Basic Crêpes
----11: Rich and Simple French Onion Soup
----12: Chicken Cordon Bleu
----13: Fondant Potato

----44: Easy Mexican Pork Burritos
----45: Chocolate Tres Leches Cake
----46: Chicken Tacos
----47: Lime Chicken Soft Tacos
----48: Quick and Easy Mango Margaritas
----49: Margaritas
----50: Chicken Adovada
----51: Instant Pot Chicken Tortilla Soup
----52: Chicken Fajita Rice Casserole
----53: Ground Beef with Homemade Taco Seasoning Mix
----54: Minute Breakfast Burrito
----55: Best Ever Ceviche
----56: Green Chicken Enchilada Soup
----57: Sheet Pan Steak and Shrimp Fajitas
----58: Churro Bundt Cake
----59: 5 Ingredient Cheesy Taco Soup
----60: Baked Breakfast Taquitos
32: Analyzing Swiss...
----1: Cheese Rösti
----2: BaumKuchen
----3: Mailaenderli
----4: Homemade Muesli
----5: Classic Swiss Fondue
----6: Swiss Almond Carrot Cake (Aargauer Rueblitorte)
----7: Anise Waffle Cookies
----8: Aargau Carrot Cake
33: Analyzing Russian...
----1: Easy Blini (Russian Pancake)
----2: Authentic Russian Salad 'Olivye'
----3: Halupki (Stuffed Cabbage)
----4: Russian Pelmeni
----5: Beef and Beet Borsc

----15: Finnish Runeberg Tortes
----16: Finnish Oven Pancakes
38: Analyzing Thai...
----1: Authentic Pad Thai
----2: Asian Coconut Rice
----3: Spicy Thai Basil Chicken (Pad Krapow Gai)
----4: The Best Thai Coconut Soup
----5: Thai Sweet Sticky Rice With Mango (Khao Neeo Mamuang)
----6: Chicken Massaman Curry
----7: Pad Thai
----8: Peanut Butter Noodles
----9: Thai Green Curry Chicken
----10: Panang Curry with Chicken
----11: Curried Coconut Chicken
----12: Thai Cucumber Salad
----13: The Best Thai Peanut Sauce
----14: Thai Pineapple Chicken Curry
----15: How to Make Beef Satay
----16: Thai Spicy Basil Chicken Fried Rice
----17: Chicken Satay
----18: How to Make Peanut Dipping Sauce
----19: Thai Steamed Mussels
----20: Thai Chicken
----21: Thai Dipping Sauce for Spring Rolls (Nam Jim Po Piah)
----22: Nong's Khao Man Gai
----23: Stir-Fried Mushrooms with Baby Corn
----24: Everything You Need to Know About Curry Paste
----25: Slow Cooker Chicken Massaman Curry
----26: Authentic Thai Green

----41: The Amazing Swedish Meatball
----42: Norwegian Potato Flatbread (Lefse)
----43: Healthier Swedish Meatballs
----44: Swedish Meatballs with Creamy Dill Sauce
----45: Fattigmann
----46: Sweet and Sour Red Cabbage
----47: Gravlox
----48: Scandinavian Almond Bars
----49: Kroppkakor - Swedish Potato Dumplings
----50: Lutefisk
----51: Fårikål
----52: Swedish Cream Wafers
----53: Swedish Hard Tack
----54: Swedish Glogg
----55: Arvidson Swedish Pancakes
----56: Norwegian Potato Klub
----57: Karelian Pies (Karjalanpiirakka)
----58: Glogi
----59: Swedish Pickled Cucumbers
----60: Torsk (Scandinavian Cod)
43: Analyzing Canadian...
----1: Nanaimo Bars
----2: Boneless Buffalo Wings
----3: Maritime Donair Sauce
----4: Gramma's Date Squares
----5: Jalapeño Jelly
----6: Real Poutine
----7: Best Fried Walleye
----8: Jamaican Style Curry Chicken
----9: French Canadian Tourtière
----10: Pure Maple Candy
----11: Dash's Donair
----12: Tourtière (French Canadian Meat Pie)
----13: Apple Cinnamon Oatm

----27: Beef Nilaga
----28: Filipino Avocado Milkshake
----29: Siopao (Filipino Steamed Dumplings)
----30: Kwek Kwek (Filipino Street Food)
----31: Chewy Coconut Bibingka (Filipino Rice Cake)
----32: Filipino Steamed Rice, Cebu Style
----33: Filipino Ribs
----34: Filipino Chicken Sopas
----35: Leche Flan
----36: Sinigang na Isda sa Miso (Fish Stew with Miso)
----37: Chicken Afritada (Filipino Stew)
----38: Garlic Rice
----39: Instant Pot Filipino Chicken Adobo
----40: Pochero
----41: Mamon (Sponge Cakes)
----42: Filipino Baked Milkfish (Baked Bangus)
----43: Biko
----44: Filipino-Style Congee (Lugaw)
----45: Coconut Sauce
----46: Paksiw na Pata (Pig's Feet Stew)
----47: Filipino Fish Escabeche
----48: Fish Sinigang (Tilapia) - Filipino Sour Broth Dish
----49: Bistek (Filipino Beef Steak)
----50: Karioka Sweet Rice Balls
----51: Philippine Longanisa de Eugenio (Sweet Sausage)
----52: Mongo Guisado (Mung Bean Soup)
----53: Authentic Filipino Pancit
----54: Filipino Corned Beef and Cabbag

In [16]:
# Cuisine with the most total number of reviews / Highest median number of reviews
# Cuisine with the highest median rating

max_reviews_count = 0
max_reviews_cuisine = ""

max_median_reviews = 0
max_median_reviews_cuisine = ""

max_median_rating = 0
max_median_rating_cuisine = ""

# Iterate through the cuisines_data dictionary
for cuisine_name, data in cuisines_data.items():
    # Total number of reviews
    total_reviews = data["Total number of reviews"]
    if total_reviews > max_reviews_count:
        max_reviews_count = total_reviews
        max_reviews_cuisine = cuisine_name

    # Median number of reviews
    median_reviews = data["Median number of reviews"]
    if median_reviews > max_median_reviews:
        max_median_reviews = median_reviews
        max_median_reviews_cuisine = cuisine_name

    # Median rating
    median_rating = data["Median rating"]
    if median_rating > max_median_rating:
        max_median_rating = median_rating
        max_median_rating_cuisine = cuisine_name
        
print(f"{max_reviews_count}: {max_reviews_cuisine}")
print(f"{max_median_reviews}: {max_median_reviews_cuisine}")
print(f"{max_median_rating}: {max_median_rating_cuisine}")

54613: Italian
248.0: Southern
4.7: Southern


In [17]:
# Write cuisine data to csv file
current_directory = Path.cwd()

def write_to_csv(cuisines_data):
    cuisine_file_name = "cuisines_data.csv"
    csv_file_path = current_directory / cuisine_file_name
    
    # Write headers
    header = ["Cuisine"] + list(list(cuisines_data.values())[0].keys())
    
    # Write data to csv
    with open(csv_file_path, mode="w", newline="") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=header)
        
        writer.writeheader()
        
        for cuisine_name, cuisine_info in cuisines_data.items():
            cuisine_info["Cuisine"] = cuisine_name
            writer.writerow(cuisine_info)

write_to_csv(cuisines_data)